# Анализ сервиса чтения книг по подписке

<ins>**Задачи проекта:**</ins>

1. Посчитайте, сколько книг вышло после 1 января 2000 года.
1. Для каждой книги посчитайте количество обзоров и среднюю оценку.
1. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.
1. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.
1. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

<ins>**Описание данных:**</ins>

**1. Таблица `books`**

    *Содержит данные о книгах:*

* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

**2. Таблица `authors`**

*Содержит данные об авторах:*

* `author_id` — идентификатор автора;
* `author` — имя автора.

**3. Таблица `publishers`**

*Содержит данные об издательствах:*

* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства;

**4. Таблица `ratings`**

*Содержит данные о пользовательских оценках книг:*

* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

**5. Таблица `reviews`**

*Содержит данные о пользовательских обзорах на книги:*

* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, написавшего обзор;
* `text` — текст обзора.

<ins>*Схема базы данных сервиса чтения книг по подписке:*</ins>
<br>
https://disk.yandex.ru/i/IUtCE5vHxFC8Kg

## Чтение базы данных

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

# устанавливаем параметры
db_config = {
            'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db' # название базы данных
            }
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

# формируем запрос и выводим данные
query = '''SELECT * FROM books LIMIT 5'''

get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Основная часть - решение задач

### Задача 1

Посчитайте, сколько книг вышло после 1 января 2000 года.

In [2]:
query = \
'''
    SELECT COUNT(*)
    FROM books AS b
    WHERE b.publication_date > '2000-01-01';
'''

get_sql_data(query)

,count
0,819


После 1 января 2000 года вышло `819` книг.

### Задача 2

Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [3]:
query = \
'''
    SELECT b.book_id,
           COUNT(DISTINCT rv.review_id) AS count_reviews,
           ROUND(AVG(rt.rating), 2) AS avg_rating
    FROM books AS b
    JOIN reviews AS rv ON b.book_id=rv.book_id
    JOIN ratings AS rt ON b.book_id=rt.book_id
    GROUP BY b.book_id;
'''

get_sql_data(query)

,book_id,count_reviews,avg_rating
0,1,2,3.67
1,2,1,2.50
2,3,3,4.67
3,4,2,4.50
4,5,4,4.00
...,...,...,...
989,996,3,3.67
990,997,3,3.40
991,998,4,3.20
992,999,2,4.50


Для каждой книги рассчитано количество обзоров в столбце `count_reviews` и средний рейтинг в столбце `avg_rating`.

### Задача 3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [4]:
query = \
'''
    SELECT pb.publisher,
           COUNT(b.book_id) AS count_books
    FROM publishers AS pb
    JOIN books AS b ON pb.publisher_id=b.publisher_id
    WHERE b.num_pages > 50
    GROUP BY pb.publisher
    ORDER BY count_books DESC
    LIMIT 1;
'''

get_sql_data(query)

,publisher,count_books
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг толще 50 страниц — `Penguin Books`, оно выпустило 42 книги.

### Задача 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [5]:
query = \
'''
    SELECT a.author,
           ROUND(AVG(rt.rating), 2) AS avg_rating
    FROM authors AS a
    JOIN books AS b ON a.author_id=b.author_id
    JOIN ratings AS rt ON b.book_id=rt.book_id    
    GROUP BY a.author_id
    HAVING COUNT(rt.rating_id) > 50
    ORDER BY AVG(rt.rating) DESC
    LIMIT 1;
'''

get_sql_data(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг, имеющих более 50 оценок - `J.K. Rowling/Mary GrandPré`. Средняя оценка его книг - 4,29.

### Задача 5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [6]:
query = \
'''
    WITH users AS (
        SELECT rt.username
        FROM ratings AS rt
        GROUP BY rt.username
        HAVING COUNT(rt.rating_id) > 48
    ),
        users_review_count AS (
        SELECT u.username,
               COUNT(rv.review_id) as review_count
        FROM users AS u
        JOIN reviews AS rv ON u.username=rv.username
        GROUP BY u.username
    )
    
    SELECT AVG(urc.review_count)
    FROM users_review_count AS urc
'''

get_sql_data(query)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - `24` обзора.

## Общие выводы (ответы к задачам)

1. После 1 января 2000 года вышло `819` книг.
1. Для каждой книги рассчитано количество обзоров в столбце `count_reviews` и средний рейтинг в столбце `avg_rating`.
1. Издательство, которое выпустило наибольшее число книг толще 50 страниц — `Penguin Books`.
1. Автор с самой высокой средней оценкой книг, имеющих более 50 оценок - `J.K. Rowling/Mary GrandPré`.
1. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - `24` обзора.